In [302]:
import _pickle as pickle
import pandas as pd
import numpy as np
from itertools import chain
import time

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


from feature_extraction import *

In [303]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [304]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [293]:
print("Estimated Runtime: 2 minutes")
start = time.time()

game_list_2013 = load_game_list('Game_List_2013.p')
player_data_2013 = pd.DataFrame.from_csv("Player_Data_2013.csv")
team_data_2013 = pd.DataFrame.from_csv("Team_Data_2013.csv")

game_list_2014 = load_game_list('Game_List_2014.p')
player_data_2014 = pd.DataFrame.from_csv("Player_Data_2014.csv")
team_data_2014 = pd.DataFrame.from_csv("Team_Data_2014.csv")

game_list_2015 = load_game_list('Game_List_2015.p')
player_data_2015 = pd.DataFrame.from_csv("Player_Data_2015.csv")
team_data_2015 = pd.DataFrame.from_csv("Team_Data_2015.csv")

game_list_2016 = load_game_list('Game_List_2016.p')
player_data_2016 = pd.DataFrame.from_csv("Player_Data_2016.csv")
team_data_2016 = pd.DataFrame.from_csv("Team_Data_2016.csv")

#features, outputs = generate_game_features(game_list_2015, player_data_2015, team_data_2015, True)

# After getting the game lists, split into training and test sets

test_set_ratio_2013 = .4
test_set_ratio_2014 = .4
test_set_ratio_2015 = .4
test_set_ratio_2016 = .4

train_games_2013, test_games_2013 = train_test_split(game_list_2013, test_size = test_set_ratio_2013)
train_games_2014, test_games_2014 = train_test_split(game_list_2014, test_size = test_set_ratio_2014)
train_games_2015, test_games_2015 = train_test_split(game_list_2015, test_size = test_set_ratio_2015)
train_games_2016, test_games_2016 = train_test_split(game_list_2016, test_size = test_set_ratio_2016)

# Change this function in feature_extraction to modify which features are used
train_features_2013, train_outputs_2013 = generate_game_features(train_games_2013, player_data_2013, team_data_2013, True)
train_features_2014, train_outputs_2014 = generate_game_features(train_games_2014, player_data_2014, team_data_2014, True)
train_features_2015, train_outputs_2015 = generate_game_features(train_games_2015, player_data_2015, team_data_2015, True)
train_features_2016, train_outputs_2016 = generate_game_features(train_games_2016, player_data_2016, team_data_2016, True)

average_features_2013, test_set_outputs_2013 = generate_average_vectors(test_games_2013, player_data_2013, team_data_2013, True)
average_features_2014, test_set_outputs_2014 = generate_average_vectors(test_games_2014, player_data_2014, team_data_2014, True)
average_features_2015, test_set_outputs_2015 = generate_average_vectors(test_games_2015, player_data_2015, team_data_2015, True)
average_features_2016, test_set_outputs_2016 = generate_average_vectors(test_games_2016, player_data_2016, team_data_2016, True)

# Split each .4 test set into half validation half test
validation_features_2013, test_features_2013, validation_outputs_2013, test_outputs_2013 = train_test_split(average_features_2013, test_set_outputs_2013, test_size=.5)
validation_features_2014, test_features_2014, validation_outputs_2014, test_outputs_2014 = train_test_split(average_features_2014, test_set_outputs_2014, test_size=.5)
validation_features_2015, test_features_2015, validation_outputs_2015, test_outputs_2015 = train_test_split(average_features_2015, test_set_outputs_2015, test_size=.5)
validation_features_2016, test_features_2016, validation_outputs_2016, test_outputs_2016 = train_test_split(average_features_2016, test_set_outputs_2016, test_size=.5)


### All the training feature and ouptuts ###
all_train_features = list(chain(train_features_2013, train_features_2014, train_features_2015, train_features_2016))
all_train_outputs = list(chain(train_outputs_2013, train_outputs_2014, train_outputs_2015, train_outputs_2016))

all_validation_features = list(chain(validation_features_2013, validation_features_2014, validation_features_2015, validation_features_2016))
all_validation_outputs = list(chain(validation_outputs_2013, validation_outputs_2014, validation_outputs_2015, validation_outputs_2016))

all_test_features = list(chain(test_features_2013, test_features_2014, test_features_2015, test_features_2016))
all_test_outputs = list(chain(test_outputs_2013, test_outputs_2014, test_outputs_2015, test_outputs_2016))


end = time.time()
print("Time Elapsed:", end-start)

Finished generating team averages
Finished generating team averages
Finished generating team averages
Finished generating team averages
Time Elapsed: 112.4008219242096


In [294]:
print(len(all_train_features), len(all_train_outputs))
print(len(all_validation_features), len(all_validation_outputs))
print(len(all_test_features), len(all_test_outputs))

print(len(all_train_features[0]), len(all_validation_features[0]), len(all_test_features[0]))

4724 4724
1580 1580
1580 1580
62 62 62


# Sklearn Classification Pipelines

In [307]:
def create_and_run_pipeline(feature_selection_model, clf_model):
    
    clf = Pipeline([
      ('feature_selection', feature_selection_model),
      ('classification', clf_model)
    ])
    
    clf.fit(all_train_features, all_train_outputs)

    print("Validation Confusion Matrix")
    print(confusion_matrix(clf.predict(all_validation_features), all_validation_outputs))
    print("Test Confusion Matrix")
    print(confusion_matrix(clf.predict(all_test_features), all_test_outputs))

    print("Number of Features Retained:", clf.named_steps['feature_selection'].get_support().tolist().count(True))
    print("Training Score:", clf.score(all_train_features, all_train_outputs))
    print("Validation Score:", clf.score(all_validation_features, all_validation_outputs))
    print("Test Score:", clf.score(all_test_features, all_test_outputs))

In [309]:
# Logistic Regression pipeline
feature_selection_model = SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))
clf_model = linear_model.LogisticRegression(penalty='l2')

create_and_run_pipeline(feature_selection_model, clf_model)

Validation Confusion Matrix
[[537 281]
 [258 504]]
Test Confusion Matrix
[[508 254]
 [277 541]]
Number of Features Retained: 42
Training Score: 1.0
Validation Score: 0.658860759494
Test Score: 0.663924050633


In [310]:
# Neural Network for classification pipeline
feature_selection_model = SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))
clf_model = MLPClassifier(solver='adam', alpha=1e-2, hidden_layer_sizes=(100,100), max_iter=200)

create_and_run_pipeline(feature_selection_model, clf_model)


Validation Confusion Matrix
[[491 240]
 [304 545]]
Test Confusion Matrix
[[455 215]
 [330 580]]
Number of Features Retained: 44
Training Score: 0.990897544454
Validation Score: 0.655696202532
Test Score: 0.655063291139


In [301]:
# Neural Network for classification pipeline
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))),
  ('classification', MLPClassifier(solver='adam', alpha=1e-2, hidden_layer_sizes=(100,100), max_iter=200))
])

clf.fit(all_train_features, all_train_outputs)

print("Number of Features Retained:", clf.named_steps['feature_selection'].get_support().tolist().count(True))
print("Training Score:", clf.score(all_train_features, all_train_outputs))
print("Validation Score:", clf.score(all_validation_features, all_validation_outputs))
print("Test Score:", clf.score(all_test_features, all_test_outputs))

Number of Features Retained: 43
Training Score: 0.990262489416
Validation Score: 0.640506329114
Test Score: 0.646202531646


In [295]:
# SVM pipeline

# Neural Network for classification pipeline
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))),
  ('classification', SVC())
])

clf.fit(all_train_features, all_train_outputs)

print("Number of Features Retained:", clf.named_steps['feature_selection'].get_support().tolist().count(True))
print("Training Score:", clf.score(all_train_features, all_train_outputs))
print("Validation Score:", clf.score(all_validation_features, all_validation_outputs))
print("Test Score:", clf.score(all_test_features, all_test_outputs))


Number of Features Retained: 42
Training Score: 1.0
Validation Score: 0.658227848101
Test Score: 0.653164556962


In [306]:
# Random Forest classifier pipeline

clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))),
  ('classification', RandomForestClassifier(max_features='log2', min_samples_leaf=10))
])

clf.fit(all_train_features, all_train_outputs)

print("Validation Confusion Matrix")
print(confusion_matrix(clf.predict(all_validation_features), all_validation_outputs))
print("Test Confusion Matrix")
print(confusion_matrix(clf.predict(all_test_features), all_test_outputs))

print("Number of Features Retained:", clf.named_steps['feature_selection'].get_support().tolist().count(True))
print("Training Score:", clf.score(all_train_features, all_train_outputs))
print("Validation Score:", clf.score(all_validation_features, all_validation_outputs))
print("Test Score:", clf.score(all_test_features, all_test_outputs))


Validation Confusion Matrix
[[519 285]
 [276 500]]
Test Confusion Matrix
[[485 266]
 [300 529]]
Number of Features Retained: 41
Training Score: 0.987298899238
Validation Score: 0.644936708861
Test Score: 0.641772151899


# Feature Selection

In [210]:
# Remove only the elements that are the same for all game instances
# Removes 2 100's for team totals, USG%
# selector = VarianceThreshold(threshold=0.0)
# test = all_train_features
# test = selector.fit_transform(test)

selector = VarianceThreshold(threshold=0.5)
test = all_train_features
test = selector.fit_transform(test)

len(test[0]), len(all_train_features[0])

(48, 64)

In [212]:
type(all_train_features)

list

In [274]:
# SVM L1 Based Feature Selection
all_train_features = np.array(all_train_features)
print(all_train_features.shape)

# C=1, 57
# C=.1, 44
# C=.01, 24
# Larger C = more values in resulting transform (less 0-weight)
lsvc = LinearSVC(C=.1, penalty="l1", dual=False).fit(all_train_features, all_train_outputs)
model = SelectFromModel(lsvc, prefit=True)

new_features = model.transform(all_train_features)

print(new_features.shape)

(4730, 64)
(4730, 43)


In [268]:
# Logistic Regression Based L1 Feature Selection
all_train_features = np.array(all_train_features)
print(all_train_features.shape)

# C=1, 64->21
# C=2, 61->20
# C=.1, 64->10
# Larger C = more values in resulting transform (less 0-weight), this is because C is INVERSE of regularization strength
lin_mod = linear_model.LogisticRegression(C=.1, penalty="l1", dual=False).fit(all_train_features, all_train_outputs)
model = SelectFromModel(lin_mod, prefit=True)

new_features = model.transform(all_train_features)

print(new_features.shape)

(4730, 64)
(4730, 10)


In [209]:
len(test[0]), len(all_train_features[0])

(62, 64)

In [207]:
test[0]

array([  38.   ,   88.   ,    0.432,    7.   ,   20.   ,    0.35 ,
         18.   ,   28.   ,    0.643,   20.   ,   35.   ,   55.   ,
         14.   ,    3.   ,    7.   ,    8.   ,   20.   ,  101.   ,
          0.503,    0.472,    0.227,    0.318,   39.2  ,   79.5  ,
         57.9  ,   36.8  ,    3.4  ,   10.9  ,    7.4  ,  115.5  ,
        109.7  ,   35.   ,   81.   ,    0.432,    6.   ,   17.   ,
          0.353,   20.   ,   27.   ,    0.741,    9.   ,   31.   ,
         40.   ,   18.   ,    5.   ,    4.   ,    7.   ,   20.   ,
         96.   ,    0.517,    0.469,    0.21 ,    0.333,   20.5  ,
         60.8  ,   42.1  ,   51.4  ,    5.7  ,    5.9  ,    7.   ,
        109.7  ,  115.5  ])

# Classification

In [157]:
### Support Vector Machines ###
svm = SVC()
svm.fit(train_features_2014, train_outputs_2014)
# svm.fit(train_features_2015, train_outputs_2015)


print("Predicted Losses - ", svm.predict(average_features_2014).tolist().count(0))
print("Predicted Wins - ", svm.predict(average_features_2014).tolist().count(1))

# Row value is actual, column value is prediction
print("2014 Confusion Matrix")
print(confusion_matrix(svm.predict(average_features_2014), test_set_outputs_2014))
print("2015 Confusion Matrix")
print(confusion_matrix(svm.predict(average_features_2015), test_set_outputs_2015))

score_2014 = svm.score(average_features_2014, test_set_outputs_2014)
score_2015 = svm.score(average_features_2015, test_set_outputs_2015)

print("2014 -", score_2014)
print("2015 -", score_2015)

Predicted Losses -  315
Predicted Wins -  301
2014 Confusion Matrix
[[211 104]
 [ 97 204]]
2015 Confusion Matrix
[[192 122]
 [110 180]]
2014 - 0.673701298701
2015 - 0.615894039735


In [158]:
### Neural Networks (Feedforward) ###

from sklearn.neural_network import MLPClassifier
clf = None
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50,50), max_iter=200)
clf.fit(train_features_2014, train_outputs_2014)
#clf.fit(train_features_2015, train_outputs_2015)

print("Predicted Losses - ", clf.predict(average_features_2014).tolist().count(0))
print("Predicted Wins - ", clf.predict(average_features_2014).tolist().count(1))

# Row value is actual, column value is prediction
print("2014 Confusion Matrix")
print(confusion_matrix(clf.predict(average_features_2014), test_set_outputs_2014))
print("2015 Confusion Matrix")
print(confusion_matrix(clf.predict(average_features_2015), test_set_outputs_2015))

score_2014 = clf.score(average_features_2014, test_set_outputs_2014)
score_2015 = clf.score(average_features_2015, test_set_outputs_2015)

print("2014 -", score_2014)
print("2015 -", score_2015)

Predicted Losses -  142
Predicted Wins -  474
2014 Confusion Matrix
[[107  35]
 [201 273]]
2015 Confusion Matrix
[[ 83  32]
 [219 270]]
2014 - 0.616883116883
2015 - 0.584437086093


In [160]:
### Logistic Regression ###

logistic_regression_classifier = linear_model.LogisticRegression(penalty='l1')
logistic_regression_classifier.fit(train_features_2015, train_outputs_2015)

# print(logistic_regression_classifier.coef_)
# for i,v in enumerate(logistic_regression_classifier.coef_[0]):
#     if v!=0:
#         print(i)

print("Predicted Losses - ", logistic_regression_classifier.predict(average_features_2014).tolist().count(0))
print("Predicted Wins - ", logistic_regression_classifier.predict(average_features_2014).tolist().count(1))

# Row value is actual, column value is prediction
print("2014 Confusion Matrix")
print(confusion_matrix(logistic_regression_classifier.predict(average_features_2014), test_set_outputs_2014))
print("2015 Confusion Matrix")
print(confusion_matrix(logistic_regression_classifier.predict(average_features_2015), test_set_outputs_2015))

score_2014 = logistic_regression_classifier.score(average_features_2014, test_set_outputs_2014)
score_2015 = logistic_regression_classifier.score(average_features_2015, test_set_outputs_2015)

print("2014 -", score_2014)
print("2015 -", score_2015)


Predicted Losses -  308
Predicted Wins -  308
2014 Confusion Matrix
[[212  96]
 [ 96 212]]
2015 Confusion Matrix
[[190 112]
 [112 190]]
2014 - 0.688311688312
2015 - 0.629139072848


In [142]:
len(team_data_2015.dropna(axis=1).columns)

36

# Regression

In [11]:
train_features_regression, train_outputs_regression = generate_game_features(train_games, player_data_2015, team_data_2015, False)

In [12]:
average_features_regression, test_set_outputs_regression = generate_average_vectors(test_games, player_data_2015, team_data_2015, False)

Finished generating team averages


In [49]:
# Standard OLS
# linearRegressionModel = linear_model.LinearRegression()
# linearRegressionModel.fit(train_features_regression, train_outputs_regression)
# print(linearRegressionModel.score(average_features_regression, test_set_outputs_regression))

# Ridge Regression
ridge_regression_model = linear_model.Ridge(alpha=1.0)
ridge_regression_model.fit(train_features_regression, train_outputs_regression)

predicted_outputs = ridge_regression_model.predict(average_features_regression)


difference = np.array(predicted_outputs) - np.array(test_set_outputs_regression)
print(np.average(difference))
print(np.std(difference))
# print(ridge_regression_model.score(average_features_regression, test_set_outputs_regression))

# Lasso Regression
# lasso_regression_model = linear_model.Lasso(alpha=1.0, max_iter=5000)
# lasso_regression_model.fit(train_features_regression, train_outputs_regression)
# print(lasso_regression_model.coef_)

# # Visualizing L1 to 0
# for i,v in enumerate(lasso_regression_model.coef_):
#     if v!=0:
#         print (i)
        
# used_data = team_data_2015.dropna(axis=1)

# print(lasso_regression_model.score(average_features_regression, test_set_outputs_regression))

1.47267603319e-14
13.0724855429
